In [1]:
#import dependencies
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
import pandas as pd
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import func


In [ ]:
# Step 1: Read CSV Files into DataFrames
industry_groups_df = pd.read_csv('Resources/Industry_Groups.csv')
top_ten_df = pd.read_csv('Resources/top_ten_with_industryID.csv')
top_ten_historic_df = pd.read_csv('Resources/top_ten_historic_5y.csv')
top_ten_historic_df.head()

In [ ]:
# #trying to see if i had any null values in 'Date' column of top_ten_historic_df
# print(top_ten_historic_df['Ticker'].isnull().sum())

In [ ]:
# top_ten_df.info()

In [ ]:
#convert datatype to date
top_ten_historic_df['Date'] = pd.to_datetime(top_ten_historic_df['Date'])


In [ ]:
top_ten_historic_df.info()

In [ ]:
# top_ten_historic_df['Date'] = pd.to_datetime(top_ten_historic_df['Date']).apply(lambda x: x.date())

In [ ]:
# print(top_ten_historic_df['Date'].unique())

In [ ]:
industry_groups_df.info()

In [ ]:
# Define Database Schema
Base = declarative_base()

class IndustryGroups(Base):
    __tablename__ = 'industry_groups'
    
    industry_name = Column(String)
    industry_id = Column(Integer, primary_key=True)
    

class TopTen(Base):
    __tablename__ = 'top_ten'
    
    ticker = Column(String, primary_key=True)
    company_name = Column(String)
    market_cap = Column(Float)
    industry_id = Column(Integer, ForeignKey('industry_groups.industry_id'))

class TopTenHistoric(Base):
    __tablename__ = 'top_ten_historic'
    
    id = Column (Integer, primary_key=True, autoincrement=True)
    Ticker = Column(String, ForeignKey('top_ten.ticker'))
    Date = Column(Date)
    Open = Column(Float)
    High = Column(Float)
    Low = Column(Float)
    Close = Column(Float)
    Adj_Close = Column('Adj Close', Float)
    Volume = Column(Integer)

#Connect to the Database and Create Tables
# Replace 'sqlite:///asx.db' with your actual database connection string
engine = create_engine('sqlite:///Resources/top_ten_asx.db')
Base.metadata.create_all(engine)


In [ ]:
#Insert DataFrames into Database
Session = sessionmaker(bind=engine)
session = Session()

# Insert IndustryGroups
session.bulk_insert_mappings(IndustryGroups, industry_groups_df.to_dict(orient="records"))

# Insert TopTen
session.bulk_insert_mappings(TopTen, top_ten_df.to_dict(orient="records"))

# Insert TopTenHistoric
session.bulk_insert_mappings(TopTenHistoric, top_ten_historic_df.to_dict(orient="records"))





In [ ]:
# #commit changes
# session.commit()

In [4]:
#import dependencies for inspector
from sqlalchemy import inspect

In [ ]:
# Instantiate an Inspector with the engine
inspector = inspect(engine)

In [ ]:
# Get a list of table names
table_names = inspector.get_table_names()
print(table_names)

In [ ]:
#Get columns of top_ten_historic
columns = inspector.get_columns('top_ten')
for column in columns:
    print(column['name'])

In [ ]:
#reflect database into ORM classes 
Base = automap_base()
Base.prepare(autoload_with=engine)
Base.classes.keys()

In [ ]:
#save a reference to the top_ten table as 'top_ten'
top_ten = Base.classes.top_ten

In [ ]:
#create a database session object
session = Session(engine)

In [ ]:
#list all of the tickers found in the top_ten table 
session.query(top_ten.ticker).all()

## Querying Existing tables

In [4]:
 # Create engine using the `top_ten_asx.db` database file
engine = create_engine('sqlite:///Resources/top_ten_asx.db')


In [5]:
#Declare a Base using automap_base()
Base = automap_base()

In [6]:
# Use the Base class to reflect the database tables
#examines the database, discover the tables and columns and relationships within the db, and set up Python classes.
Base.prepare(autoload_with=engine)

In [7]:
# Print all of the classes mapped to the Base
Base.classes.keys()

['industry_groups', 'top_ten', 'top_ten_historic']

In [8]:
#assign classes/tables to a variable
industry_groups = Base.classes.industry_groups
top_ten = Base.classes.top_ten
historic = Base.classes.top_ten_historic

In [9]:
[column for column in historic.__table__.columns]

[Column('id', INTEGER(), table=<top_ten_historic>, primary_key=True, nullable=False),
 Column('Ticker', VARCHAR(), ForeignKey('top_ten.ticker'), table=<top_ten_historic>),
 Column('Date', DATE(), table=<top_ten_historic>),
 Column('Open', FLOAT(), table=<top_ten_historic>),
 Column('High', FLOAT(), table=<top_ten_historic>),
 Column('Low', FLOAT(), table=<top_ten_historic>),
 Column('Close', FLOAT(), table=<top_ten_historic>),
 Column('Adj Close', FLOAT(), table=<top_ten_historic>),
 Column('Volume', INTEGER(), table=<top_ten_historic>)]

In [10]:
#create a session
session = Session(bind=engine)

In [11]:
#Display the row's columns and data in a dict format
first_row = session.query(historic).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x139044fa0>,
 'id': 1,
 'Date': datetime.date(2018, 11, 5),
 'High': 17.540000915527344,
 'Close': 17.149999618530273,
 'Volume': 1106127,
 'Open': 17.1299991607666,
 'Ticker': 'ARB',
 'Low': 16.3700008392334,
 'Adj Close': None}

In [12]:
# # create a new declarative base instance for defining new classes
# engine = create_engine('sqlite:///Resources/top_ten_asx.db')
# Base = declarative_base()

# class Fundamentals(Base):
#     __tablename__ = 'fundamentals'
    
#     fundamental_ID = Column(Integer, primary_key=True)
#     Ticker = Column(String, ForeignKey('top_ten.ticker'))
#     lastPrice = Column (Float)
#     Change = Column(String) 
#     Bid_Ask = Column(String) 
#     volumePerDay = Column(Float)
#     volume4wAvg= Column(Float)
#     Open = Column(Float)
#     dayRange = Column(Float)
#     prevClose = Column(String) 
#     lastTrade = Column(String)
#     oneWeek = Column(String)
#     oneMonth = Column(String)
#     YTD2023 = Column(String)
#     oneYear = Column(String)
#     vsSectorOneYr = Column(String)
#     vsASX200OneYr = Column(String)
#     marketCap = Column(Float)
#     ASXRank = Column(String)
#     sectorRank = Column(String)
#     sharesIssued = Column(Float)
#     Sector = Column(String)
#     similarCompanies = Column(String)
#     EPS = Column(Float)
#     DPS = Column(Float)
#     bookValuePerShare = Column(Float)
#     Breakdown = Column(String)
#     Recommendation = Column(String)
#     lastUpdated = Column(String)
#     PE = Column(Float)


# session = Session(engine)

# # create the table associated with the NewTable class
# Base.metadata.create_all(engine)

NoReferencedTableError: Foreign key associated with column 'fundamentals.Ticker' could not find table 'top_ten' with which to generate a foreign key to target column 'ticker'